In [1]:
!pip install category_encoders

    100% |████████████████████████████████| 61kB 18.0MB/s ta 0:00:01
You are using pip version 10.0.1, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install --upgrade pip

    100% |████████████████████████████████| 1.4MB 23.0MB/s ta 0:00:01
  Found existing installation: pip 10.0.1
    Uninstalling pip-10.0.1:
      Successfully uninstalled pip-10.0.1


In [4]:
import pandas as pd
import category_encoders as ce
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
import os
import urllib.request
import boto3
import json
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier


In [ ]:
# def download(url):
#     filename= url.split("/")[-1]
#     if not os.path.exists(filename):
#         urllib.request.urlretrieve(url, filename)

In [5]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)



In [ ]:
!pip install kaggle

In [ ]:
# download('https://www.kaggle.com/c/ds2-tree-ensembles/download/sample_submission.csv')
# download('https://www.kaggle.com/c/ds2-tree-ensembles/download/test_features.csv')
# download('https://www.kaggle.com/c/ds2-tree-ensembles/download/train_features.csv')
# download('https://www.kaggle.com/c/ds2-tree-ensembles/download/train_labels.csv')

In [ ]:
KAGGLE_COMPETITION = 'ds2-tree-ensembles'

In [ ]:
config = json.loads(open("kaggle.json").read())
os.environ['KAGGLE_USERNAME'] = config['username']
os.environ['KAGGLE_KEY'] = config['key']

In [ ]:
!kaggle competitions download -c ds2-tree-ensembles

In [ ]:
!mkdir -p data

In [ ]:
!curl https://resources.lendingclub.com/LCDataDictionary.xlsx > ./data/data-description.xlsx

In [ ]:
!unzip test_features.csv.zip

In [ ]:
!unzip train_features.csv.zip

In [ ]:
!unzip train_labels.csv.zip

In [ ]:
#change permissions

!chmod +r 'test_features.csv'
!chmod +r 'train_features.csv'
!chmod +r 'train_labels.csv'




In [5]:
X_train = pd.read_csv('train_features.csv')
X_test = pd.read_csv('test_features.csv')
y_train = pd.read_csv('train_labels.csv')['charged_off']
sample_submission = pd.read_csv('sample_submission.csv')

In [7]:
X_train.shape, X_test.shape, y_train.shape

((1309457, 103), (26724, 103), (1309457,))

In [6]:
def wrangle(X):
    X = X.copy()
    
    # Drop some columns
    X = X.drop(columns='id')  # id is random
    X = X.drop(columns=['member_id', 'url', 'desc'])  # All null
    X = X.drop(columns='title')  # Duplicative of purpose
    X = X.drop(columns='grade')  # Duplicative of sub_grade
    
    # Transform sub_grade from "A1" - "G5" to 1.1 - 7.5
    def wrangle_sub_grade(x):
        first_digit = ord(x[0]) - 64
        second_digit = int(x[1])
        return first_digit + second_digit/10
    
    X['sub_grade'] = X['sub_grade'].apply(wrangle_sub_grade)

    # Convert percentages from strings to floats
    X['int_rate'] = X['int_rate'].str.strip('%').astype(float)
    X['revol_util'] = X['revol_util'].str.strip('%').astype(float)
        
    # Transform earliest_cr_line to an integer: how many days it's been open
    X['earliest_cr_line'] = pd.to_datetime(X['earliest_cr_line'], infer_datetime_format=True)
    X['earliest_cr_line'] = pd.Timestamp.today() - X['earliest_cr_line']
    X['earliest_cr_line'] = X['earliest_cr_line'].dt.days
    
    # Create features for three employee titles: teacher, manager, owner
    X['emp_title'] = X['emp_title'].str.lower()
    X['emp_title_teacher'] = X['emp_title'].str.contains('teacher', na=False)
    X['emp_title_manager'] = X['emp_title'].str.contains('manager', na=False)
    X['emp_title_owner']   = X['emp_title'].str.contains('owner', na=False)
    
    # Drop categoricals with high cardinality
    X = X.drop(columns=['emp_title', 'zip_code'])
    
    # Transform features with many nulls to binary flags
    many_nulls = ['sec_app_mths_since_last_major_derog',
                  'sec_app_revol_util',
                  'sec_app_earliest_cr_line',
                  'sec_app_mort_acc',
                  'dti_joint',
                  'sec_app_collections_12_mths_ex_med',
                  'sec_app_chargeoff_within_12_mths',
                  'sec_app_num_rev_accts',
                  'sec_app_open_act_il',
                  'sec_app_open_acc',
                  'revol_bal_joint',
                  'annual_inc_joint',
                  'sec_app_inq_last_6mths',
                  'mths_since_last_record',
                  'mths_since_recent_bc_dlq',
                  'mths_since_last_major_derog',
                  'mths_since_recent_revol_delinq',
                  'mths_since_last_delinq',
                  'il_util',
                  'emp_length',
                  'mths_since_recent_inq',
                  'mo_sin_old_il_acct',
                  'mths_since_rcnt_il',
                  'num_tl_120dpd_2m',
                  'bc_util',
                  'percent_bc_gt_75',
                  'bc_open_to_buy',
                  'mths_since_recent_bc']

    for col in many_nulls:
        X[col] = X[col].isnull()
    
    # For features with few nulls, do mean imputation
    for col in X:
        if X[col].isnull().sum() > 0:
            X[col] = X[col].fillna(X[col].mean())
    
    # Return the wrangled dataframe
    return X


X_train = wrangle(X_train)
X_test  = wrangle(X_test)
X_train.shape, X_test.shape

((1309457, 98), (26724, 98))

In [51]:
# X_train = X_train.drop(columns=['tot_coll_amt',
# 'mths_since_recent_bc_dlq',
# 'pub_rec_bankruptcies',
# 'mths_since_recent_revol_delinq',
# 'sec_app_revol_util',
# 'mths_since_recent_inq',
# 'pub_rec',
# 'num_tl_90g_dpd_24m',
# 'mths_since_last_record',
# 'emp_title_owner',
# 'collections_12_mths_ex_med',
# 'acc_now_delinq',
# 'bc_util',
# 'chargeoff_within_12_mths',
# 'delinq_amnt',
# 'mths_since_recent_bc',
# 'num_tl_30dpd',
# 'percent_bc_gt_75',
# 'tax_liens',
# 'emp_title_teacher',
# 'bc_open_to_buy'])

In [52]:

# X_test = X_test.drop(columns=['tot_coll_amt',
# 'mths_since_recent_bc_dlq',
# 'pub_rec_bankruptcies',
# 'mths_since_recent_revol_delinq',
# 'sec_app_revol_util',
# 'mths_since_recent_inq',
# 'pub_rec',
# 'num_tl_90g_dpd_24m',
# 'mths_since_last_record',
# 'emp_title_owner',
# 'collections_12_mths_ex_med',
# 'acc_now_delinq',
# 'bc_util',
# 'chargeoff_within_12_mths',
# 'delinq_amnt',
# 'mths_since_recent_bc',
# 'num_tl_30dpd',
# 'percent_bc_gt_75',
# 'tax_liens',
# 'emp_title_teacher',
# 'bc_open_to_buy'])

In [53]:
X_train.shape, X_test.shape

((1309457, 77), (26724, 77))

In [13]:
null_counts = X_train.isnull().sum()
all(null_counts == 0)

True

In [54]:
# pipe = make_pipeline(
#     ce.OrdinalEncoder(), 
#     RandomForestClassifier(
#         n_estimators=100, 
#         class_weight='balanced', 
#         min_samples_leaf=0.005, 
#         oob_score=True, 
#         n_jobs=-1)
# )

In [12]:
# pipe = make_pipeline(
#     ce.OrdinalEncoder(), 
#     KNeighborsClassifier(n_neighbors=3,n_jobs=-1)
# )

In [17]:
pipe = make_pipeline(
    ce.OrdinalEncoder(), 
    GradientBoostingClassifier(n_estimators=1000, min_samples_leaf=0.005,max_depth=5)
)

In [19]:
pipe.fit(X_train, y_train)

KeyboardInterrupt: 

In [40]:
%%time
pipe.fit(X_train, y_train)
y_pred_proba = pipe.named_steps['randomforestclassifier'].oob_decision_function_[:, 1]
print('ROC AUC, Out-of-Bag estimate:', roc_auc_score(y_train, y_pred_proba))


KeyboardInterrupt: 

In [20]:
y_pred = pipe.predict(X_test)


In [21]:
y_pred = pd.DataFrame(y_pred)

y_pred[0].value_counts()

0    25531
1     1193
Name: 0, dtype: int64

In [22]:
sample_submission = pd.read_csv('sample_submission.csv') 


In [23]:
submission = sample_submission.copy() 


In [24]:
submission['charged_off'] = pipe.predict_proba(X_test)[:, 1] 

In [25]:
submission['charged_off'].head()

0    0.050828
1    0.156579
2    0.167891
3    0.331168
4    0.019704
Name: charged_off, dtype: float64

In [26]:
submission.to_csv('submission-007.csv', index=False)